# Modeling & Classification.




~~usar varios clusters, uno de 3grams, uno de nouns, uno de embeddings~~   
~~con esto buscar un clasificador multilabel (o simplemente agregarlos como features del modelo que le fedee al clasificador)~~  

Des métodos para clusterizar:    
-primero hago cluster separados para Tags, 2-3grams, Keys    
    luego agrego estas nuevas features al embedding, con eso puedo entrenar el clasificador.          
  
-segundo usando topic modeling q todavia no investigue     


Orden:  
Crear Espacio,  
normalizarlo,  
reducirlo,  
hacerle kmeans,  
obtener los clusters,  
medir siluete, gap, inertia, y precision con mi database.  
Agregar la medicion al df de mediciones.  


## Setup

In [40]:
import pandas as pd
import numpy as np
#import fasttext
from patterns import emoji_pattern
from gensim.models import Word2Vec, KeyedVectors
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.feature_selection import VarianceThreshold
from sentence_transformers import SentenceTransformer, util


df = pd.read_csv("db/features.csv", index_col=0)

df


,id,Q,Qless,Qlemmas,Qpos,Qkeys,Qclean,ANS,Aless,Alemmas,Apos,Akeys,Aclean,QA-keys,QA-lemmas
0,ojjgvy,"people that eat spicy food, why?",people eat spicy food,"['people', 'eat', 'spicy', 'food']","[('people', 'NNS'), ('eat', 'VBP'), ('spicy', ...",people_NNS eat_VBP spicy_NN food_NN,people that eat spicy food why,"I never liked spicy food, it just adds a pain ...",I never liked spicy food adds pain element don...,"['I', 'never', 'liked', 'spicy', 'food', 'add'...","[('I', 'PRP'), ('never', 'RB'), ('liked', 'VBD...",I_PRP never_RB liked_VBD spicy_NN food_NN add_...,I never liked spicy food it just adds a pain e...,people_NNS eat_VBP spicy_NN food_NN I_PRP nev...,"['people', 'eat', 'spicy', 'food', 'I', 'never..."
1,bc60pk,"donut or doughnut, & where are you from?",donut doughnut,"['donut', 'doughnut']","[('donut', 'NN'), ('doughnut', 'NN')]",donut_NN doughnut_NN,donut or doughnut where are you from,"donut, southern usa",donut southern usa,"['donut', 'southern', 'usa']","[('donut', 'NN'), ('southern', 'JJ'), ('usa', ...",donut_NN usa_NN,donut southern usa,donut_NN doughnut_NN donut_NN usa_NN,"['donut', 'doughnut', 'donut', 'southern', 'usa']"
2,289mmm,help finding a song from a video description?,help finding song video description,"['help', 'finding', 'song', 'video', 'descript...","[('help', 'NN'), ('finding', 'NN'), ('song', '...",help_NN finding_NN song_NN video_NN descriptio...,help finding a song from a video description,/r/tipofmytongue,rtipofmytongue,['rtipofmytongue'],"[('rtipofmytongue', 'NN')]",rtipofmytongue_NN,rtipofmytongue,help_NN finding_NN song_NN video_NN descriptio...,"['help', 'finding', 'song', 'video', 'descript..."
3,rufpr/,"i was rapedno, we had sex",rapedno sex,"['rapedno', 'sex']","[('rapedno', 'NN'), ('sex', 'NN')]",rapedno_NN sex_NN,i was rapedno we had sex,"if its not a yes, its no.",yes,['yes'],"[('yes', 'NNS')]",yes_NNS,if its not a yes its no,rapedno_NN sex_NN yes_NNS,"['rapedno', 'sex', 'yes']"
4,24vtw3,jesus h. christ what's his middle name?,jesus h christ whats middle name,"['jesus', 'h', 'christ', 'whats', 'middle', 'n...","[('jesus', 'NN'), ('h', 'NN'), ('christ', 'NN'...",jesus_NN h_NN christ_NN whats_NNS middle_VBP n...,jesus h christ whats his middle name,umm... Holy. As in Holy Christ. Isn't this com...,umm Holy As Holy Christ Isnt common knowledge,"['umm', 'Holy', 'As', 'Holy', 'Christ', 'Isnt'...","[('umm', 'JJ'), ('Holy', 'NNP'), ('As', 'IN'),...",Holy_NNP As_IN Holy_NNP Christ_NNP Isnt_NNP kn...,umm Holy As in Holy Christ Isnt this common kn...,jesus_NN h_NN christ_NN whats_NNS middle_VBP n...,"['jesus', 'h', 'christ', 'whats', 'middle', 'n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90149,oqcloa,¿what easy work is really difficult?,¿easy work really difficult,"['¿easy', 'work', 'really', 'difficult']","[('¿easy', 'JJ'), ('work', 'NN'), ('really', '...",work_NN really_RB,¿what easy work is really difficult,None by virtue of it being easy in the first p...,None virtue easy first place,"['None', 'virtue', 'easy', 'first', 'place']","[('None', 'NN'), ('virtue', 'NN'), ('easy', 'V...",None_NN virtue_NN easy_VBP place_NN,None by virtue of it being easy in the first p...,work_NN really_RB None_NN virtue_NN easy_VBP ...,"['¿easy', 'work', 'really', 'difficult', 'None..."
90150,ooswef,daddy what does nsfw mean?,daddy nsfw mean,"['daddy', 'nsfw', 'mean']","[('daddy', 'JJ'), ('nsfw', 'NNS'), ('mean', 'V...",nsfw_NNS mean_VBP,daddy what does nsfw mean,not safe for work,safe work,"['safe', 'work']","[('safe', 'JJ'), ('work', 'NN')]",work_NN,not safe for work,nsfw_NNS mean_VBP work_NN,"['daddy', 'nsfw', 'mean', 'safe', 'work']"
90151,orgpc4,"quiet people, what are your strengths?",quiet people strengths,"['quiet', 'people', 'strength']","[('quiet', 'JJ'), ('people', 'NNS'), ('strengt...",people_NNS strength_NN,quiet people what are your strengths,"Observing, attention, memory,",Observing attention memory,"['Observing', 'attention', 'memory']","[('Observing', 'VBG'), ('attent

## Modeling

podria ser capa de LSA? (a los embedings tmb?   )

In [41]:
Models = {}
gpu = False

### Tfidf Models

In [16]:
def tdidf_model(df, column, range=(2,3), min_df=20, threshold=1e-3, ngram=False):
    
    docs = df[column].fillna("")

    if ngram:
        tdidf = TfidfVectorizer(min_df=min_df, ngram_range=range)
    else:
        tdidf = TfidfVectorizer(min_df=min_df)

    #Create, Normalize and Reduce 
    model = tdidf.fit_transform(docs)
    model = normalize(model, axis=1, norm="max")
    model = VarianceThreshold(threshold=threshold).fit_transform(model)

    return np.array(model)

In [45]:
#Qkeys is a previously prunned pair of lemmas_PoS
Models["Q-keys"] = np.array(tdidf_model(df, column="Qkeys", min_df=15))

#Qless is with stopwords removed
Models["Q-ngrams"] = np.array(tdidf_model(df, column="Qless", range=(2,3), min_df=15, ngram = True))

#Question+Answer lemma_keys
Models["QA-keys"] = np.array(tdidf_model(df, column="QA-keys", min_df=15))

### Embeddings

https://kavita-ganesan.com/fasttext-vs-word2vec/

In [47]:

def w2v_doc_embedding(docs_toks):

    docs_model = []
    
    words_model = Word2Vec(docs_toks, size=300, workers=8, seed=0)

    for tokens in docs_toks:
        zero_vector = np.zeros(words_model.vector_size)
        vectors = []
        for token in tokens:
            if token in words_model.wv:
                try:
                    vectors.append(words_model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            docs_model.append(avg_vec)
        else:
            docs_model.append(zero_vector)
    return np.array(docs_model)

Models["w2v-Q"] = w2v_doc_embedding(list(df["Qlemmas"]))
Models["w2v-QA"] = w2v_doc_embedding(list(df["QA-lemmas"]))


### Pretrained Embeddings

- SBERT , glove-6B-300
    * Q, QLess, QA, QAless  


In [19]:
if gpu:
    glove60 = KeyedVectors.load_word2vec_format("glove.6B.300d.txt", binary=False, no_header=True)
    def glove_doc_embedding(docs_toks, words_model):

        docs_model = []
        

        for tokens in docs_toks:
            zero_vector = np.zeros(words_model.vector_size)
            vectors = []
            for token in tokens:
                try:
                    vectors.append(words_model[token])
                except KeyError:
                    continue
            if vectors:
                vectors = np.asarray(vectors)
                avg_vec = vectors.mean(axis=0)
                docs_model.append(avg_vec)
            else:
                docs_model.append(zero_vector)
        return docs_model
        
    Models["glove-Q"] = glove_doc_embedding(list(df["Qtoks"]), glove60)


In [20]:
if gpu:
    model = SentenceTransformer('nreimers/MiniLM-L6-H384-uncased') #
    Models["SBERT"] = model.encode(list(df["Q"].astype('str')), batch_size=70, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)


### Topic Modeling
https://github.com/ddangelov/Top2Vec
https://pythonrepo.com/repo/ddangelov-Top2Vec-python-natural-language-processing

In [21]:
from top2vec import Top2Vec
if gpu:
    Models["top2vec"] = Top2Vec(list(df["Q"].astype('str')), embedding_model='universal-sentence-encoder')

In [ ]:
from gensim.models import LdaMulticore

Models["LDA"] = lda_model = LdaMulticore(bow_corpus, 
                                   num_topics = 8,
                                   id2word = dictionary,                                    
                                   passes = 40,
                                   workers = 4)

## Clustering



Solo voy a probar con KMeans por cuestion de cantidad de datos y recursos, lo que me permite

https://scikit-learn.org/stable/modules/clustering.html
https://towardsdatascience.com/10-tips-for-choosing-the-optimal-number-of-clusters-277e93d72d92  
https://towardsdatascience.com/k-means-clustering-and-the-gap-statistics-4c5d414acd29  
https://hdbscan.readthedocs.io/en/latest/comparing_clustering_algorithms.html   s


In [53]:
import faiss


class FaissKMeans:
    def __init__(self, n_clus=10, n_init=10, max_iter=50):
        self.n_clusters = n_clus
        self.n_init = n_init
        self.max_iter = max_iter
        self.kmeans = None
        self.cluster_centers_ = None
        self.inertia_ = None

    def fit(self, X, y):
        self.kmeans = faiss.Kmeans(d=X.shape[1],
                                   k=self.n_clusters,
                                   niter=self.max_iter,
                                   nredo=self.n_init)
        self.kmeans.train(X.astype(np.float32))
        self.cluster_centers_ = self.kmeans.centroids
        self.inertia_ = self.kmeans.obj[-1]
        #print(self.kmeans.obj)

    def predict(self, X):
        return self.kmeans.index.search(X.astype(np.float32), 1)[1]

### Evaluation

inertia, silhuete, own_metric

In [54]:
#Con este dataset el metodo de elbow no se puede utiizar porque el error crece linearmente
def get_elbow(X,ran=(8,20)):
    #from kneed import KneeLocator, DataGenerator

    distortions = []
    for i in range(1000,1003):
        fkm = FaissKMeans(n_clus=i)
        fkm.fit(X, [])
        distortions.append(fkm.inertia_)
    return distortions

In [24]:
from sklearn.cluster import KMeans

n_clusters=30
kmc = KMeans(n_clusters=n_clusters,  random_state=0)
kmc.fit(X=Models["top2vec"])

df["label1"] = kmc.labels_

#vemos los primeros clusters, al ser con ngrams  la forma en la que comienza la pregunta tiene mucho peso
def print_kmc_clus(n=10):
    import random
    q_clus = [[] for i in range(n_clusters)]

    for sentence_id, cluster_id in enumerate(kmc.labels_):
        q_clus[cluster_id].append(df.iloc[sentence_id].Q)

    for i, cluster in enumerate(q_clus):
        print(f"Cluster {i+1}, len: {len(cluster)}" )
        for tw in random.sample(cluster, n):
            print(tw)
        print("\n")
        
print_kmc_clus()


SyntaxError: trailing comma not allowed without surrounding parentheses (<ipython-input-24-54ce6ed39d39>, line 1)

In [ ]:

#Compara la cantidad de aciertos con los df de test
def check_results():
    test_df = pd.read_csv("db/test_db.csv", index_col=0)
    true_df = test_df[test_df["2"]]
    false_df = test_df[test_df["2"] == False]

    n = len(true_df)
    m = len(false_df)
    same_c = 0
    for i in range(n):
        if int(df.loc[true_df.iloc[i][0]].label1) ==  int(df.loc[true_df.iloc[i][1]].label1):
            same_c += 1
    
    diff_c = 0
    for i in range(m):
        if int(df.loc[false_df.iloc[i][0]].label1) !=  int(df.loc[false_df.iloc[i][1]].label1):
            diff_c += 1
        
    return same_c/n, diff_c/m
        
check_results()
    

(0.35882828282828283, 0.7212772127721278)

## Classifying 
